# Deploy Colpali with LitServe
This notebook demonstrates how to deploy ColPali with LitServe

In [ ]:
# Import necessary libraries
import os
import torch
from PIL import Image
from colpali_engine.models import ColPali, ColPaliProcessor
import tqdm

#Enable hf_transfer for faster download
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1" # optional setting for faster dataset downloads

#Load the Model
model = ColPali.from_pretrained(
        "vidore/colpali-v1.3",
        torch_dtype=torch.bfloat16,
        device_map="mps",  # Use "cuda:0" for GPU, "cpu" for CPU, or "mps" for Apple Silicon
    ).eval()

processor = ColPaliProcessor.from_pretrained("vidore/colpali-v1.3")

In [2]:
# Your inputs
images = [
    Image.new("RGB", (32, 32), color="white"),
    Image.new("RGB", (16, 16), color="black"),
]
queries = [
    "Is attention really all you need?",
    "Are Benjamin, Antoine, Merve, and Jo best friends?",
]

# Process the inputs
batch_images = processor.process_images(images).to(model.device)
batch_queries = processor.process_queries(queries).to(model.device)

# Forward pass
with torch.no_grad():
    image_embeddings = model(**batch_images)
    query_embeddings = model(**batch_queries)

scores = processor.score_multi_vector(query_embeddings, image_embeddings)

In [ ]:
print(image_embeddings.shape)
print(query_embeddings.shape)

In [ ]:
print(scores)
print(scores.mean(axis=1))